Continuation of the student grade performance dataset
Working on the student score prediction, using the fastai python library

In [77]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
from fastai.imports import *
from fastai.structured import *

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

#import file and converting to pandas dataframe 
filename = os.path.join('../../../fastai/data/student/', 'student-mat.csv')
student_data = pd.read_csv(filename, low_memory=False, sep=',')
print('Data file shape before droping a set of columns :\n',f'{original_data.shape}')

def display_all(dataframe):
    pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000)
    display(dataframe)

Data file shape before droping a set of columns :
 (395, 33)


In [79]:
display_all(student_data.transpose())

,0,1,2,3,4,5,6,7,8,9,...,385,386,387,388,389,390,391,392,393,394
school,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP,...,MS,MS,MS,MS,MS,MS,MS,MS,MS,MS
sex,F,F,F,F,F,M,M,F,M,M,...,F,F,F,F,F,M,M,M,M,M
age,18,17,15,15,16,16,16,17,15,15,...,18,18,19,18,18,20,17,21,18,19
address,U,U,U,U,U,U,U,U,U,U,...,R,R,R,U,U,U,U,R,R,U
famsize,GT3,GT3,LE3,GT3,GT3,LE3,LE3,GT3,LE3,GT3,...,GT3,GT3,GT3,LE3,GT3,LE3,LE3,GT3,LE3,LE3
Pstatus,A,T,T,T,T,T,T,A,A,T,...,T,T,T,T,T,A,T,T,T,T
Medu,4,1,1,4,3,4,2,4,3,3,...,2,4,2,3,1,2,3,1,3,1
Fedu,4,1,1,2,3,3,2,4,2,4,...,2,4,3,1,1,2,1,1,2,1
Mjob,at_home,at_home,at_home,health,other,services,other,other,services,other,...,at_home,teacher,services,teacher,other,services,services,other,services,other
Fjob,teacher,other,other,services,other,other,other,teacher,other,other,...,other,at_home,other,services,other,services,services,other,other,at_home


In [80]:
display_all(student_data.describe(include='all').transpose())

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
school,395,2,GP,349,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sex,395,2,F,208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,395,NaN,NaN,NaN,16.6962,1.27604,15,16,17,18,22
address,395,2,U,307,NaN,NaN,NaN,NaN,NaN,NaN,NaN
famsize,395,2,GT3,281,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pstatus,395,2,T,354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Medu,395,NaN,NaN,NaN,2.74937,1.09474,0,2,3,4,4
Fedu,395,NaN,NaN,NaN,2.52152,1.0882,0,2,2,3,4
Mjob,395,5,other,141,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fjob,395,5,other,217,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
#running the first model without preprocessing the dataset, the series G2,G3 are not need so they are
#dropped from the dataframe
list_to_drop = ['G2', 'G3']
model1 = RandomForestRegressor(n_jobs=-1).fit(original_data.drop(list_to_drop, axis=1), original_data['G1'])


ValueError: could not convert string to float: 'GP'

In [82]:
#First models indicates the presences of foreign data types
train_cats(student_data)


In [83]:
display(student_data.isnull().sum().sort_index()/len(student_data))

Dalc          0.0
Fedu          0.0
Fjob          0.0
G1            0.0
G2            0.0
G3            0.0
Medu          0.0
Mjob          0.0
Pstatus       0.0
Walc          0.0
absences      0.0
activities    0.0
address       0.0
age           0.0
failures      0.0
famrel        0.0
famsize       0.0
famsup        0.0
freetime      0.0
goout         0.0
guardian      0.0
health        0.0
higher        0.0
internet      0.0
nursery       0.0
paid          0.0
reason        0.0
romantic      0.0
school        0.0
schoolsup     0.0
sex           0.0
studytime     0.0
traveltime    0.0
dtype: float64

In [84]:
features, label, _ = proc_df(student_data.drop(list_to_drop[0], axis=1), 'G1')

The shuffling of data into training set, test set and validaton sets hasn't been carried out, running the model to see it's general performance

In [85]:
#New model after preprocessing 
student_model = RandomForestRegressor(n_jobs=-1).fit(features, label)
score = student_model.score(features, label)

print('Model score accuracy :\n{:.5f}'.format(score))

Model score accuracy :
0.97163
